# verificar as equipes na escala de serviço e horários

In [ ]:
import tabula
lista_tabelas = tabula.read_pdf("EscalaServico.pdf", pages="all")
print(len(lista_tabelas))
for tabela in lista_tabelas:
    display(tabela)


In [ ]:
import tabula
lista_tabelas = tabula.read_pdf("EscalaServico.pdf", pages="all")
print(len(lista_tabelas))
for tabela in lista_tabelas:
    display(tabela)
